In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import pickle as p
import networkx as nx
import matplotlib.pyplot as plt
from ActorNetwork import ActorNetwork
import time
import numpy as np
import time, sys, pickle
from collections import defaultdict, deque

min_year=2020
max_year = 2026
basics = pd.read_csv(
    "data/title.basics.tsv",
    sep="\t",
    dtype=str,
    na_values="\\N",
    usecols=["tconst", "titleType", "primaryTitle", "originalTitle", "startYear"]
)

basics = basics[basics["titleType"].isin(["movie", "tvMovie"])].copy()
basics["startYear"] = pd.to_numeric(basics["startYear"], errors="coerce")

basics = basics[
    (basics["startYear"] >= min_year) &
    (basics["startYear"] <= max_year)
].copy()

basics["primary_norm"] = basics["primaryTitle"]
basics["original_norm"] = basics["originalTitle"]

print("IMDb basics rows after filter:", len(basics))


IMDb basics rows after filter: 134268


In [ ]:
# -----------------------------
# Load IMDb AKAS (filtered)
# -----------------------------
akas = pd.read_csv(
    "data/title.akas.tsv",
    sep="\t",
    dtype=str,
    na_values="\\N",
    usecols=["titleId", "title"]
)

akas = akas[akas["titleId"].isin(set(basics["tconst"]))].copy()
akas["aka_norm"] = akas["title"]

print("IMDb akas rows after filter:", len(akas))

# -----------------------------
# Load CREW + NAMES (unchanged)
# -----------------------------
crew = pd.read_csv("data/title.crew.tsv", sep="\t", dtype=str, na_values="\\N")
names = pd.read_csv("data/name.basics.tsv", sep="\t", dtype=str, na_values="\\N")

names["name_norm"] = names["primaryName"]
name_map = dict(zip(names["nconst"], names["name_norm"]))

# Build tconst → directors map
director_map = {}
crew["directors"] = crew["directors"].fillna("")
for tconst, dstr in zip(crew["tconst"], crew["directors"]):
    director_map[tconst] = [name_map[d] for d in dstr.split(",") if d in name_map]

# -----------------------------
# FAST Title Index Construction
# -----------------------------
title_index = {}

def add(k, v):
    if k:
        title_index.setdefault(k, set()).add(v)

# Primary + original titles
for t, tc in zip(basics["primary_norm"], basics["tconst"]):
    add(t, tc)

for t, tc in zip(basics["original_norm"], basics["tconst"]):
    add(t, tc)

# Alternate titles
for t, tc in zip(akas["aka_norm"], akas["titleId"]):
    add(t, tc)

print("Title index size:", len(title_index))

IMDb akas rows after filter: 674704


In [ ]:
crew

In [ ]:
names

In [ ]:
names["knownForTitles"].apply(lambda x: repr(x)).value_counts().head(10)


In [ ]:
#names = names[names["knownForTitles"].isna()]
names = names[
   ~names["knownForTitles"].isna() &
    ~names["knownForTitles"].astype(str).str.strip().eq("")
]
names

In [ ]:
basics

In [ ]:
basics["titleType"].unique()

In [ ]:
basics = basics[basics["titleType"]=="movie"]




In [ ]:
basics

In [ ]:
movie_tconsts = set(
    basics.loc[basics["titleType"] == "movie", "tconst"]
)
def known_titles_all_in_basics(kft):
    if pd.isna(kft):
        return False
    titles = kft.split(",")
    return all(t in movie_tconsts for t in titles)
names_filtered = names[names["knownForTitles"].apply(known_titles_all_in_basics)]


In [ ]:
names_filtered

In [ ]:
#basics["startYear"]==2020
#movies2020 = basics[basics["startYear"]==2020]

In [ ]:
#movies2020

In [ ]:
#crew

In [ ]:
# =========================
# BUILD MOVIE → ACTORS LOOKUP
# =========================
movie_to_actors = defaultdict(deque)

for _, row in names_filtered.iterrows():
    if pd.isna(row["knownForTitles"]):
        continue
    for movie in row["knownForTitles"].split(","):
        movie_to_actors[movie].append(row["nconst"])

In [ ]:
movie_to_actors

In [ ]:
an=ActorNetwork()
counter = 0

for movie, actors in movie_to_actors.items():
    if len(actors) > 1:   # skip single-actor movies
        an.addMovie(movie, deque(actors))
        counter += 1

        if counter % 20000 == 0:
            print(f"{counter} movies processed")

In [ ]:
an